In [ ]:
# Imports ||||Run on Restart|||

from utils import get_embeddings_from_state_dict, seed_everything, prepare_tsne_data, test_model, pickle_loader

import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
from scipy import stats
import yaml

import torch
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"

from glycowork.glycan_data.loader import lib
from glycowork.ml.models import SweetNet
from glycowork.ml.processing import dataset_to_dataloader

# Statistics Silo

This is where you pipe in the data generated by the HBBS or ABBS to compare results and get statistics.

In [ ]:
# --- Parameters to set before running the analyses below ---

BASE_RANDOM_STATE = 42  # Initial seed for reproducibility of the entire sequence of experiments
DIR = "C:/OneDrive/New_base_1.1" # Path to the directory containing the models in relation to this 
STAT_DIR = "../Statistics_and_Plots" # Path to the directory where statistical data and plots will be saved
os.makedirs(STAT_DIR, exist_ok=True) # Create the directory if it doesn't exist
EXPERIMENT = "species_family" # Name of the experiment to test (for single experiment analysis like euclidian and t-SNE)
NUM_RUNS = 10 # Number of runs of experiment
RUN_NUMBER = 7 # Used for t-SNE and other analyzes that take a single run
NORMALIZE = False # Normalize the embeddings before t-SNE or Euclidean distance analysis


In [ ]:
# --- Load and Prepare All Summary Results from CSV files in DIR ---

print(f"Loading experiment summaries from: {DIR}")


all_experiment_summaries = []

# Iterate through all files in the directory
# Check if the file name starts with "summary_" and ends with ".csv"
for item_name in os.listdir(DIR):
    if item_name.startswith("summary_") and item_name.endswith(".csv"):
        summary_file_path = os.path.join(DIR, item_name)
        experiment_name = item_name[len("summary_"): -len(".csv")] 

        try:
            df_exp_summary = pd.read_csv(summary_file_path)
            df_exp_summary['experiment_name'] = experiment_name
            all_experiment_summaries.append(df_exp_summary)
            print(f"Loaded: {item_name}")

        except Exception as e:
            print(f"Could not load {item_name}: {e}")


# Concatenate all DataFrames into a single DataFrame
df_combined_results = pd.concat(all_experiment_summaries, ignore_index=True)

# Melt the DataFrame from wide to long format 
id_vars = ['experiment_name']
metric_cols = [col for col in df_combined_results.columns if '_loss' in col or '_lrap' in col or '_ndcg' in col or '_time' in col]

df_melted_results = df_combined_results.melt(id_vars=id_vars, 
                                             value_vars=metric_cols,
                                             var_name='metric_full_name', 
                                             value_name='value')

# Extract the metric type and model configuration from the metric_full_name
df_melted_results[['model_config', 'metric_type']] = df_melted_results['metric_full_name'].str.rsplit('_', n=1, expand=True)

print("You now have a bunch of experminental data ready for analysis!")

## Euclidian distance analysis
Created on a whim. 
This allows you to compare euclidian distances (that annoying copilot autocomplete thing suggested it below code I was writing for t-SNE analysis and I got intrigued and let it finish the code. after editing the code to be actually useful I did some research. seems like this analysis could actually be useful for my project).

### Simple framework for tests

In [ ]:
# Simple comparison framework to compare the embeddings of different models

# Construct paths to the relevant state dictionaries
baseline_trainable_path = f"{DIR}/saved_models_{EXPERIMENT}/baseline_trainable_run_{RUN_NUMBER}_state_dict.pth"
untrained_baseline_trainable_path = f"{DIR}/untrained_models_{EXPERIMENT}/baseline_trainable_run_{RUN_NUMBER}_state_dict.pth"
baseline_fixed_path = f"{DIR}/saved_models_{EXPERIMENT}/baseline_fixed_run_{RUN_NUMBER}_state_dict.pth"
untrained_baseline_fixed_path = f"{DIR}/untrained_models_{EXPERIMENT}/baseline_fixed_run_{RUN_NUMBER}_state_dict.pth"

# Load the embeddings from the state dictionaries\n",
baseline_emb = get_embeddings_from_state_dict(baseline_trainable_path)
untrained_baseline_emb = get_embeddings_from_state_dict(untrained_baseline_trainable_path)
fixed_baseline_emb = get_embeddings_from_state_dict(baseline_fixed_path)
untrained_fixed_baseline_emb2 = get_embeddings_from_state_dict(untrained_baseline_fixed_path)

# Calculate the Euclidean distance between the embeddings\n",
fixed_trained_euclidean_distance = np.linalg.norm(baseline_emb - fixed_baseline_emb)
fixed_untrained_euclidean_distance = np.linalg.norm(baseline_emb - untrained_baseline_emb)
trainable_untrained_euclidean_distance = np.linalg.norm(fixed_baseline_emb - untrained_fixed_baseline_emb2)


# Print the Euclidean distance values
print(f"Euclidean distance between baseline trainable and fixed embeddings: {fixed_trained_euclidean_distance}")
print(f"Euclidean distance between baseline trainable and untrained baseline embeddings: {fixed_untrained_euclidean_distance}")
print(f"Euclidean distance between fixed baseline and untrained fixed baseline embeddings: {trainable_untrained_euclidean_distance}")

Reproducability\untrained_models_kingdom_test1\baseline_fixed_run_2_state_dict.pth

### Proper analytical framwork running on all runs

In [ ]:
# --- Generate a dictionary of lists containing the results of euclidian distance analysis of all embeddings of the experiment specified above ---

# Set the random seed for reproducibility
seed_everything(BASE_RANDOM_STATE, silent=True)


# Initialize a dictionary to store the results
Euclidian_results = {"baseline_euclidean_distance": [],
                    "glylm_euclidean_distance": [],
                    "baseline_glylm_euclidean_distance": [],
                    "baseline_runs_euclidean_distance": []}

print(f"Calculating the Euclidean distance of the embeddings of the experiment {EXPERIMENT} with {NUM_RUNS} runs...")
print()

for runs in range(NUM_RUNS):
    # --- load the embeddings ---
    # Construct paths to the relevant state dictionaries
    other_run = runs + 2 if runs < NUM_RUNS - 1 else 1 
    baseline_trainable_path = f"{DIR}/saved_models_{EXPERIMENT}/baseline_trainable_run_{runs+1}_state_dict.pth"
    baseline_trainable_comp_path = f"{DIR}/saved_models_{EXPERIMENT}/baseline_trainable_run_{other_run}_state_dict.pth"
    baseline_fixed_path = f"{DIR}/untrained_models_{EXPERIMENT}/baseline_trainable_run_{runs+1}_state_dict.pth"
    infused_trainable_path = f"{DIR}/saved_models_{EXPERIMENT}/infused_trainable_run_{runs+1}_state_dict.pth"
    infused_fixed_path = f"{DIR}/saved_models_{EXPERIMENT}/infused_fixed_run_{runs+1}_state_dict.pth"

    # Load the embeddings from the state dictionaries
    baseline_emb = get_embeddings_from_state_dict(baseline_trainable_path)
    baseline_comp_emb = get_embeddings_from_state_dict(baseline_trainable_comp_path)
    fixed_baseline_emb = get_embeddings_from_state_dict(baseline_fixed_path)
    glylm_emb = get_embeddings_from_state_dict(infused_fixed_path)
    trained_glylm_emb = get_embeddings_from_state_dict(infused_trainable_path)

    # run L2 normalization on the embeddings
    if NORMALIZE:
        baseline_emb = baseline_emb / np.max(np.linalg.norm(baseline_emb, axis=1, keepdims=True))
        baseline_comp_emb = baseline_comp_emb / np.max(np.linalg.norm(baseline_comp_emb, axis=1, keepdims=True))
        fixed_baseline_emb = fixed_baseline_emb / np.max(np.linalg.norm(fixed_baseline_emb, axis=1, keepdims=True))
        glylm_emb = glylm_emb / np.max(np.linalg.norm(glylm_emb, axis=1, keepdims=True))
        trained_glylm_emb = trained_glylm_emb / np.max(np.linalg.norm(trained_glylm_emb, axis=1, keepdims=True))

    # Calculate the Euclidean distance between the embeddings
    baseline_euclidean_dist = np.linalg.norm(baseline_emb - fixed_baseline_emb)
    glylm_euclidean_dist = np.linalg.norm(glylm_emb - trained_glylm_emb)
    baseline_glylm_euclidean_dist = np.linalg.norm(baseline_emb - glylm_emb)
    # Calculate the Euclidean distance between the embeddings of two runs
    baseline_runs_euclidean_dist = np.linalg.norm(baseline_emb - baseline_comp_emb)

    # add the euclidean distance of the current run to the list of results
    Euclidian_results["baseline_euclidean_distance"].append(baseline_euclidean_dist)
    Euclidian_results["glylm_euclidean_distance"].append(glylm_euclidean_dist)
    Euclidian_results["baseline_glylm_euclidean_distance"].append(baseline_glylm_euclidean_dist)
    Euclidian_results["baseline_runs_euclidean_distance"].append(baseline_runs_euclidean_dist)

# --- Calculate the average of the euclidean distances ---
baseline_euclidean_dist_avg = np.mean(Euclidian_results["baseline_euclidean_distance"])
glylm_euclidean_dist_avg = np.mean(Euclidian_results["glylm_euclidean_distance"])
baseline_glylm_euclidean_dist_avg = np.mean(Euclidian_results["baseline_glylm_euclidean_distance"])
baseline_runs_euclidean_dist_avg = np.mean(Euclidian_results["baseline_runs_euclidean_distance"])

# Calculate the standard deviation of the euclidean distances
baseline_euclidean_dist_std = np.std(Euclidian_results["baseline_euclidean_distance"])
glylm_euclidean_dist_std = np.std(Euclidian_results["glylm_euclidean_distance"])
baseline_glylm_euclidean_dist_std = np.std(Euclidian_results["baseline_glylm_euclidean_distance"])
baseline_runs_euclidean_dist_std = np.std(Euclidian_results["baseline_runs_euclidean_distance"])

# Print the average euclidean distance values with their standard deviation
print(f"Average Baseline (fixed to trainable) Euclidean Distance: {baseline_euclidean_dist_avg:.2f} ± {baseline_euclidean_dist_std:.2f}")
print(f"Average GlyLM (fixed to trainable) Euclidean Distance: {glylm_euclidean_dist_avg:.3f} ± {glylm_euclidean_dist_std:.3f}")
print(f"Average Baseline-GlyLM [trainable] Euclidean Distance: {baseline_glylm_euclidean_dist_avg:.3f} ± {baseline_glylm_euclidean_dist_std:.3f}")
print(f"Average Baseline [trainable] (comparing runs) Euclidean Distance: {baseline_runs_euclidean_dist_avg:.3f} ± {baseline_runs_euclidean_dist_std:.3f}")



#### Before  normalization
```
Calculating the Euclidean distance of the embeddings of the experiment Kingdom1 with 10 runs...

Average Baseline (fixed to trainable) Euclidean Distance: 1281.91 ± 0.83 <-- ignore this, that comparison was flawed
Average GlyLM (fixed to trainable) Euclidean Distance: 30.51 ± 1.66
Average Baseline-GlyLM [trainable] Euclidean Distance: 1217.21 ± 0.54
Average Baseline [trainable] (comparing runs) Euclidean Distance: 1282.16 ± 1.21
```
#### After normalization
```
Calculating the Euclidean distance of the embeddings of the experiment Kingdom1 with 10 runs...

Average Baseline (fixed to trainable) Euclidean Distance needs to be calculated with the untrained baseline
Average GlyLM (fixed to trainable) Euclidean Distance: 5.23 ± 0.99
Average Baseline-GlyLM [trainable] Euclidean Distance: 61.78 ± 1.04
Average Baseline [trainable] (comparing runs) Euclidean Distance: 60.21 ± 1.00
```

## t-SNE Analysis

In [ ]:
# --- Loading the embeddings of the specified run number for t-SNE ---

# Construct paths to the relevant state dictionaries for the specified run number
baseline_trained_path = f"{DIR}/saved_models_{EXPERIMENT}/baseline_trainable_run_{RUN_NUMBER}_state_dict.pth"
baseline_fixed_path = f"{DIR}/untrained_models_{EXPERIMENT}/baseline_trainable_run_{RUN_NUMBER}_state_dict.pth"
infused_trained_path = f"{DIR}/saved_models_{EXPERIMENT}/infused_trainable_run_{RUN_NUMBER}_state_dict.pth"
infused_fixed_path = f"{DIR}/saved_models_{EXPERIMENT}/infused_fixed_run_{RUN_NUMBER}_state_dict.pth"

# comparing different runs of the same model
other_run = RUN_NUMBER + 1 if RUN_NUMBER != 5 else 1 

baseline_trained_comp_path = f"{DIR}/saved_models_{EXPERIMENT}/baseline_trainable_run_{other_run}_state_dict.pth"
baseline_fixed_comp_path = f"{DIR}/untrained_models_{EXPERIMENT}/baseline_trainable_run_{other_run}_state_dict.pth"
infused_trained_comp_path = f"{DIR}/saved_models_{EXPERIMENT}/infused_trainable_run_{other_run}_state_dict.pth"

# Load the embeddings from the state dictionaries
baseline_trained_emb = get_embeddings_from_state_dict(baseline_trained_path)
baseline_fixed_emb = get_embeddings_from_state_dict(baseline_fixed_path)
infused_trained_emb = get_embeddings_from_state_dict(infused_trained_path)
infused_fixed_emb = get_embeddings_from_state_dict(infused_fixed_path)

baseline_trained_comp_emb = get_embeddings_from_state_dict(baseline_trained_comp_path)
baseline_fixed_comp_emb = get_embeddings_from_state_dict(baseline_fixed_comp_path)
infused_trained_comp_emb = get_embeddings_from_state_dict(infused_trained_comp_path)

# --- Construct a dictionary of the lists of embedding arrays and labels for prepare_tsne_data ---
all_tsne_embeddings = [

# All embeddings for t-SNE
{'name' : 'all', 'embeddings' : [baseline_trained_emb, baseline_fixed_emb, infused_trained_emb, infused_fixed_emb],
'labels' : ["Baseline (trained)", "Baseline (original)", "GlyLM (trained)", "GlyLM (original)"]},
# Baseline comparison before and after training
{'name' : 'baseline', 'embeddings' : [baseline_trained_emb, baseline_fixed_emb],
'labels' : ["Baseline (trained)", "Baseline (original)"]},
# Infused comparison before and after training
{'name' : 'infused', 'embeddings' : [infused_trained_emb, infused_fixed_emb],
'labels' : ["Infused (trained)", "Infused (original)"]},
# Baseline vs Infused comparison
{'name' : 'baseline_infused', 'embeddings' : [baseline_trained_emb, infused_trained_emb],
'labels' : ["Baseline (trained)", "Infused (trained)"]},
# Just the Baseline embeddings
{'name' : 'base', 'embeddings' : [baseline_trained_emb],
'labels' : ["Baseline embeddings after training"]},
# Just the Infused embeddings
{'name' : 'GlyLM', 'embeddings' : [infused_trained_emb],
'labels' : ["GlyLM embeddings"]},
# Baseline comparison between two runs
{'name' : 'baseline_comp', 'embeddings' : [baseline_trained_comp_emb, baseline_trained_emb],
'labels' : ["Baseline (run 1)", "Baseline (run 2)"]},
# Infused comparison between two runs
{'name' : 'infused_comp', 'embeddings' : [infused_trained_comp_emb, infused_trained_emb],
'labels' : ["Infused (run 1)", "Infused (run 2)"]},
# Baseline Fixed comparison between two runs
{'name' : 'baseline_fixed_comp', 'embeddings' : [baseline_fixed_comp_emb, baseline_fixed_emb],
'labels' : ["Fixed Baseline (run 1)", "Fixed Baseline (run 2)"]},
# Add more comparisons as needed
]   


# Prepare the data for t-SNE
for tsne_target in all_tsne_embeddings:
    tsne_target['embeddings'], tsne_target['labels'] = prepare_tsne_data(
        tsne_target['embeddings'], tsne_target['labels'], normalize=NORMALIZE
    )


In [ ]:
# Quick loader to just look at one set of embeddings

embeddings = get_embeddings_from_state_dict("../Shit/Embedding_experiments_3/saved_models_kingdom_emb1/baseline_trainable_run_1_state_dict.pth")
test_name = "embedding 1"


tsne_input_array, tsne_labels = prepare_tsne_data(
    embedding_arrays=[embeddings], # Pass 'embeddings' as a list
    embedding_names=[test_name], # Pass 'test_name' as a list
    normalize=NORMALIZE
)

# Initialize t-SNE
tsne = TSNE(n_components=3, 
            random_state=BASE_RANDOM_STATE, 
            perplexity=30, 
            learning_rate=200,
            max_iter=1000)


# Pass the directly obtained tsne_input_array to fit_transform
tsne_coords = tsne.fit_transform(tsne_input_array)

print("t-SNE coordinates generated successfully.")

In [ ]:
# --- t-SNE analysis ---
# possible tsne_target_ids: baseline_infused, infused, baseline, base, GlyLM, all 
# Comparison between runs of the same model: baseline_comp, infused_comp, baseline_fixed_comp

tsne_target_id = "infused"
for tsne_target in all_tsne_embeddings:
    if tsne_target['name'] == tsne_target_id:
        tsne_embeddings = tsne_target['embeddings']
        tsne_labels = tsne_target['labels']
        break
else:
    raise ValueError(f"Target '{tsne_target_id}' not found in the list of embeddings.")



# Initialize t-SNE
tsne = TSNE(n_components=2, # change to 3 for 3D visualization
            random_state=BASE_RANDOM_STATE, 
            perplexity=30, # experiment between 5 and 50
            learning_rate=300, # experiment between 100 and 1000
            max_iter=1000) # experiment between 1000 and 5000
# Experiment with perplexity, learning_rate, and n_iter later for optimal visualization.
# (create a function to do this automatically)

# Initialize the t-SNE model and fit it to the embeddings
tsne_coords = tsne.fit_transform(tsne_embeddings)

glycoword_base_list = sorted(lib.keys()) 
glycoword_base_list.append('Ontology')
num_concatenated_sets = tsne_coords.shape[0] // len(glycoword_base_list)
tsne_glycowords = glycoword_base_list * num_concatenated_sets

In [ ]:
# --- Plot t-SNE scatterplot or density plot (or both) ---
# Settings
density = False # Set to True for density plot
scatter = True # Set to True for scatter plot
glycoword_annotate = False # Set to True to annotate glycowords on the plot
snfg_only = True # Set to True to only plot SNFG monosaccharides  (if glycoword_annotate is True)

# Glycowords to annotate on the plot
glycowords_to_annotate = [
    'Gal', 'GlcNAc', 'Man', 'Fuc', 'Neu5Ac', 'Glc', 
]

print(f't-SNE Visualization of {tsne_target_id.replace("_", " ").title()} Embeddings of {EXPERIMENT.replace("_", " ").title()} Experiment')

# --- Plotting the t-SNE results ---
# Settings for the plots
plt.figure(figsize=(10, 10)) # Adjust figure size as needed
# Set the color palette
sns.set_palette("colorblind") # Choose a color palette (e.g., 'viridis', 'tab10', 'Paired', 'colorblind', etc.)
# Set the font size for the plot
plt.rcParams.update({'font.size': 14}) # Adjust font size for better readability


# Generate a DataFrame for the t-SNE coordinates and labels
plot_df = pd.DataFrame(tsne_coords, columns=['x', 'y'])
plot_df['Embedding Source'] = tsne_labels
plot_df['Glycoword'] = tsne_glycowords 

# SNFG monosaccharides
snfg_glycowords = [
    '4eLeg', '6dAlt', '6dAltNAc', '6dGul', '6dTal', '6dTalNAc', 'Abe', 'Aci', 'All', 'AllA',
    'AllN', 'AllNAc', 'Alt', 'AltA', 'AltN', 'AltNAc', 'Api', 'Ara', 'Bac', 'Col',
    'DDmanHep', 'Dha', 'Dig', 'Fru', 'Fuc', 'FucNAc', 'Gal', 'GalA', 'GalN', 'GalNAc',
    'Glc', 'GlcA', 'GlcN', 'GlcNAc', 'Gul', 'GulA', 'GulN', 'GulNAc', 'Ido', 'IdoA',
    'IdoN', 'IdoNAc', 'Kdn', 'Kdo', 'Leg', 'LDmanHep', 'Lyx', 'Man', 'ManA', 'ManN',
    'ManNAc', 'Mur', 'MurNAc', 'MurNGc', 'Neu', 'Neu5Ac', 'Neu5Gc', 'Oli', 'Par', 'Pse',
    'Psi', 'Qui', 'QuiNAc', 'Rha', 'RhaNAc', 'Rib', 'Sia', 'Sor', 'Tag', 'Tal',
    'TalA', 'TalN', 'TalNAc', 'Tyv', 'Xyl'
]

# If snfg_only is True, filter the data to only include SNFG monosaccharides
if snfg_only:
    # Filter plot_df to include only rows where 'Glycoword' is in the SNFG set
    plot_df_filtered = plot_df[plot_df['Glycoword'].isin(snfg_glycowords)].copy()

    # Update the lists for annotation/symbolization to only include filtered glycowords
    # This ensures warnings aren't printed for non-SNFG glycowords
    glycowords_to_annotate_filtered = [word for word in glycowords_to_annotate if word in snfg_glycowords]
    #glycowords_to_symbolize_filtered = {word: marker for word, marker in glycowords_to_symbolize.items() if word in snfg_glycowords}

    # Use the filtered DataFrame and filtered lists for plotting
    plot_df = plot_df_filtered # Use the filtered DataFrame from here on
    glycowords_to_annotate = glycowords_to_annotate_filtered
    #glycowords_to_symbolize = glycowords_to_symbolize_filtered


if density:
    sns.kdeplot(
        x='x', y='y',
        hue='Embedding Source',
        data=plot_df,
        fill=True, # Fill the density areas
        alpha=0.5, # Transparency for density areas
        linewidth=0, # Remove line borders for smoother fills
        gridsize=100, # Number of grid points for density estimation
    )

if scatter:
    sns.scatterplot(
        x='x', y='y',
        hue='Embedding Source', # Color points by their source (Trained Baseline, Raw GlyLM, Trained Infused)
        data=plot_df,
        legend='brief', #'auto', 'brief', 'full', or False
        alpha=0.5, # Transparency for overlapping points
        s=20, # Size of points
        #edgecolor='black' # edge color for better visibility
    )
if glycoword_annotate:
    for glycoword in glycowords_to_annotate:
        # Find all points corresponding to this glycoword
        glycoword_points = plot_df[plot_df['Glycoword'] == glycoword]
        
        if not glycoword_points.empty:
            # For simplicity, let's annotate the first occurrence if multiple, or average
            # To avoid clutter, you might pick only one point per glycoword to annotate if many exist
            # Or, plot them all with smaller text/different style
            
            # Example: Annotate the first instance of the glycoword found
            x_coord = glycoword_points['x'].iloc[0]
            y_coord = glycoword_points['y'].iloc[0]
            
            plt.annotate(glycoword, (x_coord, y_coord),
                        textcoords="offset points", # Offset text from point
                        xytext=(5,5), # Offset by (5,5) pixels
                        ha='center', # Horizontal alignment
                        arrowprops=dict(arrowstyle='-', connectionstyle='arc3,rad=.2', linewidth=0.5))
        else:
            print(f"Warning: Glycoword '{glycoword}' not found in the t-SNE data to annotate.")



#plt.title('t-SNE Visualization of Glycan Embeddings')
plt.title(f't-SNE Visualization of {tsne_target_id.replace("_", " ").title()} Embeddings of {EXPERIMENT.replace("_", " ").title()} Experiment', fontsize=16)
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
# Fine-tune legend position
plt.legend(title='Embedding Source', loc='upper right', bbox_to_anchor=(1, 1), 
           ncol=1, fontsize = 14, edgecolor='black', framealpha=0.8, markerscale = 2,
           markerfirst = True, fancybox=True, draggable = True)
# move the legend outside the plot
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
#plt.grid(True, linestyle='--', alpha=0.6) # Add a subtle grid
plt.show()




In [ ]:
# Plot 3D t-SNE scatterplot

# --- Imports needed for 3D plotting (ensure these are at the top of your notebook) ---
import matplotlib.pyplot as plt # <--- ADD THIS IMPORT
from mpl_toolkits.mplot3d import Axes3D # For 3D plotting
import pandas as pd
import numpy as np # <--- ADD THIS IMPORT (often used for plot utilities like colormaps)

# --- Plotting Code ---
fig = plt.figure(figsize=(12, 10)) # Adjust figure size as needed
ax = fig.add_subplot(111, projection='3d') # Create a 3D subplot

# Create a DataFrame for easier handling of colors/labels with matplotlib
plot_df_3d = pd.DataFrame(tsne_coords, columns=['x', 'y', 'z'])
plot_df_3d['Embedding Source'] = tsne_labels

# Get unique labels for coloring
unique_labels = plot_df_3d['Embedding Source'].unique()
colors = plt.cm.get_cmap('viridis', len(unique_labels)) # Use a colormap for distinct colors

# Plot each point
for i, label in enumerate(unique_labels):
    subset = plot_df_3d[plot_df_3d['Embedding Source'] == label]
    ax.scatter(subset['x'], subset['y'], subset['z'],
               color=colors(i),
               label=label,
               alpha=0.6,
               s=20) # Size of points

ax.set_title(f'3D t-SNE Visualization of {test_name} Embeddings') # Use test_name for title
ax.set_xlabel('t-SNE Component 1')
ax.set_ylabel('t-SNE Component 2')
ax.set_zlabel('t-SNE Component 3')

ax.legend(title='Embedding Source', bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

plt.show() # <--- ADD THIS LINE TO DISPLAY THE PLOT

## Statistical Analysis of Metrics

In [ ]:
# --- Define Statistical Comparisons ---

# This will define the statistical comparisons you want to perform.
# Each entry specifies a 'metric_type' to analyze and the 'groups' to compare.
# Possible metric types: 'lrap', 'time', 'loss', 
# possible configs: baseline_trainable, baseline_fixed, infused_trainable, infused_fixed

statistical_comparisons = [
    {
        'name': 'Infused_vs_Baseline_LRAP',
        'metric_type': 'lrap',
        'group1_config': 'infused_trainable',
        'group2_config': 'baseline_trainable',
        'description': 'Comparison of Infused (trainable) vs. Baseline (trainable) LRAP.'
    },
    {
        'name': 'Baseline_Fixed_vs_Trainable_LRAP',
        'metric_type': 'lrap',
        'group1_config': 'baseline_trainable',
        'group2_config': 'baseline_fixed',
        'description': 'Comparison of Baseline (trainable) vs. Baseline (fixed) LRAP.'
    },
    {
        'name': 'Infused_Fixed_vs_Trainable_LRAP',
        'metric_type': 'lrap',
        'group1_config': 'infused_trainable',
        'group2_config': 'infused_fixed',
        'description': 'Comparison of Infused (trainable) vs. Infused (fixed) LRAP.'
    },
    {
        'name': 'Infused_vs_Baseline_Time', 
        'metric_type': 'time',
        'group1_config': 'infused_trainable',
        'group2_config': 'baseline_trainable',
        'description': 'Comparison of Infused (trainable) vs. Baseline (trainable) training time.'
    },
    # Add more comparisons here as needed, 
]

print(f"Defined {len(statistical_comparisons)} statistical comparisons to run.")

In [ ]:
# --- Quick comparison of the results of the different Experiments using just baseline trainable and one metric ---
metric_quick = 'lrap' # Choose the metric to analyze (e.g., 'lrap', 'loss', 'ndcg' 'time')
configuration = 'infused_trainable' # baseline_trainable, infused_trainable, etc.
# Filter the melted DataFrame for LRAP results
df_lrap_results = df_melted_results[df_melted_results['metric_type'] == metric_quick]
df_lrap_results = df_lrap_results[df_lrap_results['model_config'] == configuration]
df_lrap_results = df_lrap_results.groupby(['experiment_name'])['value'].agg(['mean', 'std', 'count']).reset_index()

print(f"Quick comparison of the {metric_quick} results for {configuration} of the different experiments:")
print(df_lrap_results)



min_class_size = 2
Quick comparison of the lrap results for baseline_trainable of the different experiments:
       experiment_name      mean       std  count
0  disease_association  0.842077  0.012655     10
2      species_kingdom  0.959948  0.001940     10
3        tissue_sample  0.745605  0.021077     10

vs min_class_size = 32
Quick comparison of the lrap results for baseline_trainable of the different experiments:
            experiment_name      mean       std  count
0    disease_assoc_e5_min32  0.918973  0.012601     10
1  species_kingdom_e5_min32  0.964062  0.002279     10
2    tissue_sample_e1_min32  0.827414  0.020074     10

In [ ]:
# --- Generate Performance Tables (Mean ± Std) for a Specific Metric ---

# Settings
table_metric = 'lrap' # Choose the metric to analyze ('lrap', 'loss', 'ndcg' 'time')
table_name = f"Core table({table_metric})" # Name of the table to be generated

# Filter data for the chosen metric type
df_filtered_metric = df_melted_results[df_melted_results['metric_type'] == table_metric]

# Aggregate mean and std
aggregated_stats = df_filtered_metric.groupby(['model_config', 'experiment_name', 'metric_type'])['value'].agg(['mean', 'std']).reset_index()

# Format 'mean' and 'std' into a single 'mean ± std' string
aggregated_stats['mean_std_string'] = aggregated_stats.apply(
    lambda row: f"{row['mean']:.3f} ± {row['std']:.3f}", axis=1
)

# Pivot the table
pivoted_table = aggregated_stats.pivot_table(
    index='experiment_name',
    columns='model_config',
    values='mean_std_string',
    aggfunc='first'
)

# Print the table
print(f"\n--- Generated Table ---")
print(pivoted_table) 

# Save the table to CSV
output_filename = os.path.join(STAT_DIR, f"{table_name}.csv")
pivoted_table.to_csv(output_filename, index=True) 
print(f"\nTable saved to: {output_filename}")

In [ ]:
# ---- t-Test Statistical Comparisons ----


stat_comp_name = "Base_stats" # statistical comparison file name

# Initialize a list to store t-test results
t_test_results = []

# Iterate through the defined statistical comparisons
for comp_def in statistical_comparisons:
    metric_type = comp_def['metric_type']
    group1_config = comp_def['group1_config']
    group2_config = comp_def['group2_config']

    # Filter data for the specific metric type
    df_metric = df_melted_results[df_melted_results['metric_type'] == metric_type]

    # Per-Experiment Comparisons
    for experiment in df_metric['experiment_name'].unique():
        data_g1 = df_metric[(df_metric['experiment_name'] == experiment) & 
                            (df_metric['model_config'] == group1_config)]['value']
        data_g2 = df_metric[(df_metric['experiment_name'] == experiment) & 
                            (df_metric['model_config'] == group2_config)]['value']

        t_stat, p_value = stats.ttest_ind(data_g1, data_g2)
        
        t_test_results.append({
            'comparison_name': comp_def['name'],
            'metric_type': metric_type,
            'experiment_name': experiment,
            'group1': group1_config,
            'group2': group2_config,
            'mean_g1': data_g1.mean(),
            'std_g1': data_g1.std(),
            'mean_g2': data_g2.mean(),
            'std_g2': data_g2.std(),
            't_statistic': t_stat,
            'p_value': p_value,
            'significant_p_05': p_value < 0.05
        })

    # Overall Comparison
    overall_data_g1 = df_metric[df_metric['model_config'] == group1_config]['value']
    overall_data_g2 = df_metric[df_metric['model_config'] == group2_config]['value']

    t_stat_overall, p_value_overall = stats.ttest_ind(overall_data_g1, overall_data_g2)
    
    t_test_results.append({
        'comparison_name': comp_def['name'],
        'metric_type': metric_type,
        'experiment_name': 'Overall',
        'group1': group1_config,
        'group2': group2_config,
        'mean_g1': overall_data_g1.mean(),
        'std_g1': overall_data_g1.std(),
        'mean_g2': overall_data_g2.mean(),
        'std_g2': overall_data_g2.std(),
        't_statistic': t_stat_overall,
        'p_value': p_value_overall,
        'significant_p_05': p_value_overall < 0.05
    })

# Convert results to DataFrame and save
df_t_test_summary = pd.DataFrame(t_test_results)
output_filename_t_test = os.path.join(STAT_DIR, f"{stat_comp_name}.csv")
df_t_test_summary.to_csv(output_filename_t_test, index=False)
print(f"\nStatistical comparison results saved to: {output_filename_t_test}")

# Print head of the results DataFrame
print("\n--- T-Test Summary Results Head ---")
print(df_t_test_summary.head())

## Test set Tests


In [ ]:
# --- Find Best Model ---
# you need to run the Load and Prepare All Summary Results from CSV files in DIR cell before this section to have df_melted_results ready.

# Define the metric and strategy for finding the best model
metric_to_find_best_by = 'lrap' # 'lrap', 'loss', 'ndcg', 'time'
strategy_for_best = 'max' # 'max' for LRAP/NDCG, 'min' for Loss/Time

# --- Perform the search ---

# Filter for the specific metric type (no filtering by model_config_types)
df_filtered_by_metric = df_melted_results[df_melted_results['metric_type'] == metric_to_find_best_by].copy()

# Find the row with the best metric value across ALL configurations
if strategy_for_best == 'max':
    best_row_info = df_filtered_by_metric.loc[df_filtered_by_metric['value'].idxmax()]
else: # strategy_for_best == 'min'
    best_row_info = df_filtered_by_metric.loc[df_filtered_by_metric['value'].idxmin()]

# Extract and print the details of the best model
best_experiment_name = best_row_info['experiment_name']
best_model_config = best_row_info['model_config'] # This will now be the specific config found
best_metric_value = best_row_info['value']
original_row_index_in_combined_df = best_row_info.name
run_number_within_experiment = (original_row_index_in_combined_df % NUM_RUNS) + 1

print(f"\n--- Overall Best Model Found (by {metric_to_find_best_by} - {strategy_for_best}) ---")
print(f"  Best model was found in configuration: {best_model_config}")
print(f"  Experiment: {best_experiment_name}")
print(f"  Best {metric_to_find_best_by.upper()} Value: {best_metric_value:.4f}")
print(f"  Run Number within Experiment: {run_number_within_experiment}/{NUM_RUNS}")


In [ ]:
# --- Run test tester on best model from experiment ---

# Construct relevant paths
model_state_path = os.path.join(DIR, f"saved_models_{best_experiment_name}",
                                f"{best_model_config}_run_{run_number_within_experiment}_state_dict.pth")
test_set_path = os.path.join(DIR, f"test_set_{best_experiment_name}.pkl")
experiment_settings_path = os.path.join(DIR, f"experiment_settings_{best_experiment_name}.yaml")


# Load Experiment Settings to get Model Architecture Parameters
with open(experiment_settings_path, 'r') as f:
    exp_settings_for_model = yaml.safe_load(f)

num_classes = exp_settings_for_model['num_classes']
batch_size = exp_settings_for_model['global_parameters_for_this_experiment'].get('batch_size') 
    
    

# Instantiate SweetNet Model and Load State Dict 
lib_size_for_model = len(lib)
model_best = SweetNet(lib_size=lib_size_for_model, num_classes=num_classes, hidden_dim=320)
model_best.to(device) 

model_best.load_state_dict(torch.load(model_state_path, map_location=device))

''' Using old data that saved the whole dataset, I should actually revert that change and keep this commented out 
test_data = pickle_loader(test_set_path)
test_glycans = test_data['test_glycans']
test_labels = test_data['test_labels']
test_dataloader = dataset_to_dataloader(glycan_list = test_glycans, 
                                        labels = test_labels,
                                        batch_size = batch_size,
                                        drop_last = False,
                                        augment_prob = 0, 
                                        generalization_prob = 0
                                        )
'''

test_dataloader =  pickle_loader(test_set_path)

# --- Evaluate the Best Model on the Test Set ---
test_metrics = test_model(model_best, test_dataloader, num_classes)

print("\n--- Final Test Set Performance for Overall Best Model ---")
print(f"  Test Loss: {test_metrics['loss']:.4f}")
print(f"  Test LRAP: {test_metrics['lrap']:.4f}")
print(f"  Test NDCG: {test_metrics['ndcg']:.4f}")
